<a href="https://colab.research.google.com/github/IsadoraPrevitalle/LSTM_paraClassificacaoDeTexto/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #Codificação da classe
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences #token com mesmo tamanho
from google.colab import files
import numpy as np

In [ ]:
!pip install keras
import keras.utils
from keras.utils import to_categorical
!pip install keras.preprocessing
from keras_preprocessing.sequence import pad_sequences

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.3 MB/s eta 0:00:00


In [ ]:
files.upload()

In [ ]:
Tweets = pd.read_csv("Tweets.csv")
Tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


Analise exploratória

In [ ]:
Tweets.groupby(['airline_sentiment']).size()

airline_sentiment
negative    9178
neutral     3099
positive    2363
dtype: int64

In [ ]:
#Filtragem
#Pegar dados com maior confiança - Dados com maior qualidade
Tweets = Tweets[Tweets['airline_sentiment_confidence'] > 0.8]
print(Tweets)

                 tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                           1.0   
3      570301031407624196          negative                           1.0   
4      570300817074462722          negative                           1.0   
5      570300767074181121          negative                           1.0   
9      570295459631263746          positive                           1.0   
...                   ...               ...                           ...   
14631  569588464896876545          negative                           1.0   
14633  569587705937600512          negative                           1.0   
14636  569587371693355008          negative                           1.0   
14637  569587242672398336           neutral                           1.0   
14638  569587188687634433          negative                           1.0   

               negativereason  negativereason_confidence         airline  \

In [ ]:
#dados com forma numérica e tabular
token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values) #Atributo text, onde está comentário

In [ ]:
# Gerar tokens
# Completar tokens com 0 à direita (padronizar tamanho - tabulação)
X = token.texts_to_sequences(Tweets['text'].values)  # Atributo text, onde está comentário user
X = pad_sequences(X, padding='post', maxlen=100)

# Transformar 3 classes em dados numéricos
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['airline_sentiment'])
print(y)
# classes de saída em formato numérico
y = to_categorical(y, num_classes=3)  # Especificar o número de classes

# Divisão dos dados em treinamento e teste
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)
print(y_treinamento)

[1 0 0 ... 0 1 0]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [ ]:
# Empilhar camadas da RNP
modelo = Sequential()
modelo.add(Embedding(input_dim=len(token.word_index)+1, output_dim=128, input_length=X.shape[1]))
modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation='tanh', recurrent_activation='sigmoid', unroll=False, use_bias=True))
modelo.add(Dense(units=3, activation='softmax'))

# Compilar o modelo
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelo.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 128)          1638784   
                                                                 
 spatial_dropout1d_5 (Spati  (None, 100, 128)          0         
 alDropout1D)                                                    
                                                                 
 lstm_5 (LSTM)               (None, 196)               254800    
                                                                 
 dense_5 (Dense)             (None, 3)                 591       
                                                                 
Total params: 1894175 (7.23 MB)
Trainable params: 1894175 (7.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# Treinar o modelo
modelo.fit(X_treinamento, y_treinamento, batch_size=30, epochs=10, verbose=1, validation_data=(X_teste, y_teste))

Epoch 1/10
245/245 [==============================] - 88s 348ms/step - loss: 0.8281 - accuracy: 0.7003 - val_loss: 0.7908 - val_accuracy: 0.7192
Epoch 2/10
245/245 [==============================] - 81s 330ms/step - loss: 0.8200 - accuracy: 0.7014 - val_loss: 0.8110 - val_accuracy: 0.7192
Epoch 3/10
245/245 [==============================] - 77s 312ms/step - loss: 0.8216 - accuracy: 0.7014 - val_loss: 0.7926 - val_accuracy: 0.7192
Epoch 4/10
245/245 [==============================] - 78s 319ms/step - loss: 0.8197 - accuracy: 0.7014 - val_loss: 0.7891 - val_accuracy: 0.7192
Epoch 5/10
245/245 [==============================] - 79s 321ms/step - loss: 0.8188 - accuracy: 0.7014 - val_loss: 0.7946 - val_accuracy: 0.7192
Epoch 6/10
245/245 [==============================] - 73s 297ms/step - loss: 0.8189 - accuracy: 0.7014 - val_loss: 0.7954 - val_accuracy: 0.7192
Epoch 7/10
245/245 [==============================] - 73s 297ms/step - loss: 0.8191 - accuracy: 0.7014 - val_loss: 0.7905 - val_ac

In [ ]:
#Mini previsão para avaliar performance

loss, accuracy = modelo.evaluate(X_teste, y_teste)
print('Loss: ', loss)
print('Accuracy: ', accuracy)

99/99 [==============================] - 9s 93ms/step - loss: 0.7912 - accuracy: 0.7192
Loss:  0.7912194132804871
Accuracy:  0.7192479372024536


In [ ]:
#Realizando previsões
prev = modelo.predict(X_teste)
print(prev)

99/99 [==============================] - 9s 81ms/step
[[0.6836459  0.1604604  0.15589373]
 [0.6836459  0.1604604  0.15589373]
 [0.6836459  0.1604604  0.15589373]
 ...
 [0.6836459  0.16046043 0.15589373]
 [0.6836459  0.16046043 0.15589373]
 [0.6836459  0.1604604  0.15589373]]


Vader - Baseado em regras

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
mas = SentimentIntensityAnalyzer()

In [ ]:
a = 'I hate this movie! But I loved the girl'
x = mas.polarity_scores(a)
print(x)

{'neg': 0.181, 'neu': 0.385, 'pos': 0.434, 'compound': 0.6476}


In [ ]:
print(type(x))

<class 'dict'>


In [ ]:
print(x['compound'])

0.6476


In [ ]:
from textblob import TextBlob
teste = TextBlob('I Love NY!')
print(teste.sentiment)

Sentiment(polarity=0.625, subjectivity=0.6)


In [ ]:
!pip install translate

In [ ]:
from translate import Translator

In [ ]:
translator = Translator(from_lang='pt', to_lang='en')
traducao = translator.translate('O filme era bom')
print(traducao)

The movie was good


In [ ]:
x = mas.polarity_scores(traducao)
print(x)

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}
